In [ ]:
# !pip install wandb

In [1]:
import numpy as np
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms.v2 as transforms

In [2]:
wandb.login()
# 3dc8367198d0460ba99efb94e713de7e299e685d

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3], [3,5,5,7,7], [7,7,5,5,3], [3,5,7,9,11] ]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'lr': {
            'values': [0.0003, 0.001, 0.003]
        },
        'activation': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        'optimizer': {
            'values': ['adam', 'nadam']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_aug': {
            'values': ['true','false']
        },
        'batch_size': {
            'values': [32, 64, 128]
        },
        'num_dense':{
            'values': [64, 128, 256]
        }
    }
}

optimizers = {
    'adam': optim.Adam,
    'nadam': optim.NAdam
}


sweep_id = wandb.sweep(sweep=sweep_config, project='Ass_2')

Create sweep with ID: 00v3pk4s
Sweep URL: https://wandb.ai/arun_cs23m017/Ass_2/sweeps/00v3pk4s


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 10
img_size = 256

# Define the CNN architecture
class CNN(nn.Module):
    def __init__(self, config, num_classes=10):
        super(CNN, self).__init__()
        self.config = config
        self.img_size = 256
        self.in_channels = 3
        self.num_classes = 10
        self.num_epochs = 10
        self.learning_rate = self.config.lr
        self.batch_size = self.config.batch_size
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0).to(device)
        self.bn1d = nn.BatchNorm1d(self.config.num_dense).to(device)
        self.dropout = nn.Dropout(p=self.config.dropout).to(device)
        self.fc2 = nn.Linear(self.config.num_dense, num_classes).to(device)

        # Load and transform the data
        transform = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])
        
        transform_aug = transforms.Compose([
            transforms.Resize((256,256)),
            transforms.RandomRotation(degrees=30),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        if(self.config.data_aug == 'true'):
            self.train_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=transform_aug)
#             self.test_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=transform_aug)
        else:
            self.train_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=transform)
        
        self.test_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=transform) 
        self.train_dataset, self.val_dataset = torch.utils.data.random_split(self.train_dataset, [8000, 1999])
        self.train_loader = torch.utils.data.DataLoader(dataset=self.train_dataset, batch_size=self.batch_size, shuffle=True)
        self.val_loader = torch.utils.data.DataLoader(dataset=self.val_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_loader = torch.utils.data.DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=True)
        
        if self.config.activation == 'relu':
            self.activation = F.relu
        elif self.config.activation == 'gelu':
            self.activation = F.gelu
        elif self.config.activation == 'silu':
            self.activation = F.silu
        else:
            self.activation = F.mish
      
    def forward(self, x):
        self.in_channels = 3
        self.wh = img_size
        self.padding = 0
        self.stride = 1
        
        for i in range(5):
            self.conv = nn.Conv2d(self.in_channels, self.config.filt_org[i], self.config.kernel_size[i], self.stride, self.padding).to(device)
            self.bn2d = nn.BatchNorm2d(self.config.filt_org[i]).to(device)
            x = self.conv(x)
            x = self.activation(x)
            if(self.config.batch_norm == 'true'):
                x = self.bn2d(x)
            x = self.pool(x)
            self.wh = (((self.wh - self.config.kernel_size[i] + 2*self.padding) // self.stride) + 1) // 2
            x_shape = self.config.filt_org[i] * self.wh * self.wh
            self.in_channels = self.config.filt_org[i]
        x = x.view(-1,x_shape)
        self.fc1 = nn.Linear(x_shape, self.config.num_dense).to(device)
        x = self.activation(self.fc1(x))
        if(self.config.batch_norm == 'true'):
            x = self.bn1d(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    def accuracy(self, model, criterion, loader):
        correct = 0
        total = 0
        total_loss = 0
        with torch.no_grad():
            for data in loader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                loss = criterion(outputs, labels)
                total_loss += loss.item() * self.batch_size

        acc = correct / total
        total_loss /= total
        return acc, total_loss
    
    def train(self, model, criterion, optimizer):
        total_step = len(self.train_loader)
        for epoch in range(self.num_epochs):
            print(epoch)
            correct = 0
            train_loss = 0
            for i, (images, labels) in enumerate(self.train_loader):
                images, labels = images.to(device), labels.to(device)
                # Forward pass 
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum().item()
                
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

                if (i+1)%10 == 0:
                    print('Epoch [{}/{}], Step [{}/{}], Avg Loss: {:.4f}'.format(epoch+1, self.num_epochs, i+1, total_step, train_loss/(i+1)))
            train_acc = correct / (total_step * self.batch_size) 
            train_loss /= total_step
            val_acc, val_loss = self.accuracy(model, criterion, self.val_loader)
            print(train_acc, train_loss,
                  val_acc, val_loss, "\n")
            wandb.log({'train_accuracy': train_acc})
            wandb.log({'train_loss': train_loss})
            wandb.log({'val_accuracy': val_acc})
            wandb.log({'val_loss': val_loss})

In [ ]:
def main():
    with wandb.init() as run:
        model = CNN(wandb.config, num_classes).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optimizers[wandb.config.optimizer](model.parameters(), lr=wandb.config.lr)
        model.train(model, criterion, optimizer)
        print(model.accuracy(model, criterion, model.test_loader))

wandb.agent(sweep_id, function=main, count=10) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: xzqgrih1 with config:
wandb: 	activation: silu
wandb: 	batch_norm: true
wandb: 	batch_size: 64
wandb: 	data_aug: false
wandb: 	dropout: 0
wandb: 	filt_org: [128, 128, 64, 64, 32]
wandb: 	kernel_size: [3, 5, 7, 9, 11]
wandb: 	lr: 0.001
wandb: 	num_dense: 256
wandb: 	optimizer: nadam
wandb: Currently logged in as: cs23m017 (arun_cs23m017). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(
